# Open-source LLMs as Agents with `ChatHuggingFace`


Open source LLMs are becoming viable general purpose agents.

The goal of this notebook is to demonstrate how to make use of open-source LLMs as chat models to enable their usage and experimentation with agent-based workflows.

We use [Hugging Face Inference Endpoints](https://huggingface.co/docs/inference-endpoints/index) with [LangChain's `ChatHuggingFace`]().

In particular, we will:
1. Utilize the [HuggingFaceEndpoint](https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/llms/huggingface_endpoint.py) (or [HuggingFaceTextGenInference](https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/llms/huggingface_text_gen_inference.py) or [HuggingFaceHub](https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/llms/huggingface_hub.py)) integration to call a [HF Inference Endpoint](https://huggingface.co/inference-endpoints) that's serving an LLM via [Text Generation Inference (TGI)](https://huggingface.co/docs/text-generation-inference/index)
2. Utilize the `ChatHuggingFace` class that interfaces between LangChain's [Chat Messages](https://python.langchain.com/docs/modules/model_io/chat/#messages) and the hosted LLM by leveraging [Hugging Face's Chat Templates](https://huggingface.co/docs/transformers/chat_templating) to power a `ChatAgent` pipeline.
4. Demonstrate how to use an open-source LLM in a zero-shot ReAct Agent workflow.
5. Understand how several different open-source LLM's perform as general purpose agents by running an asynchronous evaluation pipeline using LLM as the judge. 


> Note: To run this notebook, you'll need to have:
> - an LLM deployed via a Hugging Face Inference Endpoint (the LLM must have a `chat_template` defined in its `tokenizer_config.json`)
> - A Hugging Face Token with access to the deployed endpoint saved as an environment variable: `HUGGINGFACEHUB_API_TOKEN`
> - A SerpAPI key saved as an environment variable: `SERPAPI_API_KEY`
> - An OpenAI API key saved as an environment variable: `OPENAI_API_KEY`

## Setup

In [ ]:
!pip install -q --upgrade transformers langchain text-generation python-dotenv jinja2 langchainhub numexpr datasets tqdm openai sentencepiece protobuf matplotlib google-search-results

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from dotenv import load_dotenv
import numpy as np
import pandas as pd
import glob
from tqdm.notebook import tqdm
import plotly.express as px
from datasets import load_dataset


from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents import AgentExecutor
from langchain.agents.output_parsers import (
    ReActJsonSingleInputOutputParser,
)
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.agents import load_tools
from langchain.tools.render import render_text_description_and_args
from langchain.chat_models import ChatOpenAI

from prompts import SYSTEM_PROMPT, HUMAN_PROMPT, EVALUATION_PROMPT_TEMPLATE
from evaluation import build_evaluator, evaluate_answers
from run_agents import run_full_tests
from agents import build_hf_agent, build_openai_agent

load_dotenv(override=True)

## 1. Instantiate an LLM

#### `HuggingFaceHub`

In [ ]:
from langchain.llms.huggingface_hub import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    task="text-generation",
    model_kwargs={
        "max_new_tokens": 512,
        "top_k": 50,
        "temperature": 0.1,
        "repetition_penalty": 1.03,
    },
)

## 2. Create a wrapper for `BaseChatModel` to apply chat templates

In [ ]:
from langchain.schema import HumanMessage
from langchain_community.chat_models.huggingface import ChatHuggingFace

Instantiate the model and some messages to pass.

In [ ]:
messages = [
    HumanMessage(
        content="You're a helpful assistant. What happens when an unstoppable force meets an immovable object?"
    ),
]

chat_model = ChatHuggingFace(llm=llm)
chat_model.model_id

Call the model.

In [ ]:
res = chat_model.invoke(messages)
print(res.content)

## Tests

Here we'll test out our model as a zero-shot ReAct Agent.
Configure the agent with a `react-json` style prompt and access to a search engine and calculator.

### Define tools

In [ ]:
TOOLS = load_tools(["serpapi", "llm-math"], llm=llm)
# Rename tools in the same format used by other tools
TOOLS[0].name = "search"
TOOLS[1].name = "calculator"

In [ ]:
TOOLS[0]

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        HumanMessagePromptTemplate.from_template(
            SYSTEM_PROMPT + "\nSo, here is my question:" + HUMAN_PROMPT
        ),
    ]
)
prompt = prompt.partial(
    tool_description=render_text_description_and_args(TOOLS),
    tool_names=", ".join([t.name for t in TOOLS]),
)

# define the agent
chat_model_with_stop = chat_model.bind(stop=["\nObservation"])
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
    }
    | prompt
    | chat_model_with_stop
    | ReActJsonSingleInputOutputParser()
)

# instantiate AgentExecutor
agent_executor = AgentExecutor(
    agent=agent,
    tools=TOOLS,
    verbose=True,
    return_intermediate_steps=True,
    handle_parsing_errors=True,
    max_iterations=5,
)

In [ ]:
example = {"input": "What is the age of Leonardo diCaprio, raised to the power 0.43?"}

out = agent_executor.invoke(example)

# Evaluation Experiment 

- Find a group of test questions that use a certain set of tools to solve (HotpotQA)
- Run several different models as agent to solve the test questions (OS and proprietary)
- Use a LLM as a judge
- Use OS models as a judge
- Report correlations

### Evaluation dataset

In [ ]:
full_eval_dataset = load_dataset("m-ric/agents_small_benchmark", split="train")

### Run tests

In [ ]:
import json
import glob

for file in glob.glob("output/*.json"):
    output = json.load(open(file, "r"))
    print(file)
    print(len(output))

    output = [
        el
        for el in output
        if el["question"] in full_eval_dataset["question"]
        and not (el["agent_error"] is not None and "inference API" in el["agent_error"])
    ]

    print(len(output))

    json.dump(output, open(file, "w"))

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
agent_endpoints = {
    # "Zephyr-7b-beta": "https://ytjpei7t003tedav.us-east-1.aws.endpoints.huggingface.cloud",
    # "Mixtral-8x7B-Instruct-v0.1": "https://iw8z8uxlp03gvuxc.us-east-1.aws.endpoints.huggingface.cloud",
    # "OpenHermes-2.5-Mistral-7B": "https://ou70xe634aa21gsc.us-east-1.aws.endpoints.huggingface.cloud"
    # "SOLAR-10.7B": "https://zu17gw7sl8axqe8u.us-east-1.aws.endpoints.huggingface.cloud",
    # 'Llama-2-70b-chat': 'https://xwabjkpbpoqtnd4a.us-east-1.aws.endpoints.huggingface.cloud',
}

agents = {name: build_hf_agent(endpoint) for name, endpoint in agent_endpoints.items()}

# uncomment below to test GPT as an agent
# agents["GPT4"] = build_openai_agent(model_id="gpt-4-1106-preview")
# agents["GPT3.5"] = build_openai_agent(model_id="gpt-3.5-turbo-1106")

# run eval
await run_full_tests(dataset=full_eval_dataset, agents=agents)
print("Question answering is complete!")

# Evaluate with LLM-as-a-judge

In [ ]:
eval_chat_model = ChatOpenAI(model="gpt-4-1106-preview", temperature=0)
eval_model_name = "GPT4"

In [ ]:
for file in tqdm(glob.glob("output/*.json")):
    evaluate_answers(
        file,
        eval_chat_model,
        eval_model_name,
        EVALUATION_PROMPT_TEMPLATE,
    )
print("Evaluation is complete!")

## Visualize results

In [ ]:
result_df = pd.concat([pd.read_json(f) for f in glob.glob("output/*.json")])
result_df["no_prediction"] = result_df["prediction"].apply(lambda x: True if x is None else False)
result_df["has_agent_error"] = result_df["agent_error"].apply(
    lambda x: True if isinstance(x, str) else False
)


def interpret_result(x):
    try:
        return int(x) - 1
    except:
        return 0


result_df["eval_score_Mixtral"] = result_df["eval_score_Mixtral"].apply(interpret_result)

# Override results with human evaluation if there is one
result_df["eval_score_GPT4"] = result_df.apply(
    lambda row: row["eval_score_human"]
    if not np.isnan(row["eval_score_human"])
    else row["eval_score_GPT4"],
    axis=1,
)
result_df["eval_score_GPT4"] = result_df["eval_score_GPT4"].apply(interpret_result)

result_df["task"] = result_df["task"].apply(lambda x: ("HotpotQA" if "HotpotQA" in x else x))
result_df.groupby(["agent_name", "task"]).agg(
    {
        "parsing_error": "sum",
        "iteration_limit_exceeded": "sum",
        "no_prediction": "sum",
        "has_agent_error": "sum",
    }
)

In [ ]:
table_result = result_df.groupby(["agent_name", "task"], sort=False)[["eval_score_GPT4"]].mean()
table_result = table_result / 4 * 100  # set results in percentage
display(table_result.unstack())

In [ ]:
# Assert that all agents were tried on all questions
# np.testing.assert_array_equal(
#     result_df["agent_name"].value_counts().values,
#     [100] * len(result_df["agent_name"].value_counts()),
# )

table_result = table_result.reset_index()
sorter = [
    "GPT4",
    "Mixtral-8x7B-Instruct-v0.1",
    "GPT3.5",
    "Zephyr-7b-beta",
    "Llama-2-70b-chat",
    "OpenHermes-2.5-Mistral-7B",
    "SOLAR-10.7B",
]
table_result = table_result.sort_values(
    "agent_name", key=lambda column: column.map(lambda e: sorter.index(e))
)
# Plot results
fig = px.bar(
    table_result,
    x="agent_name",
    y="eval_score_GPT4",
    color="task",
    title=f"Average Evaluation Score (LLM-as-a-judge)",
    labels={
        "agent_name": "Agent",
        "task": "Task",
        "score": "Performance",
        "eval_score_GPT4": "Score",
    },
)
fig.update_layout(
    width=1000,
    height=600,
    barmode="group",
    yaxis_range=[0, 100],
)
fig.update_traces(texttemplate="%{y:.0f}", textposition="outside")
fig.layout.yaxis.ticksuffix = "%"
fig.show()

### Study intermediate steps

In [ ]:
try:
    result_df["tools_used"] = result_df["intermediate_steps"].apply(
        lambda row: ([step["tool"] for step in row] if row is not None else None)
    )
except:
    pass
result_df["has_answer"] = result_df["prediction"].apply(lambda row: (row is not None))
result_df_answers_only = result_df.loc[result_df["has_answer"] == True]
result_df_answers_only["correct_answer"] = (
    result_df_answers_only[f"eval_score_{eval_model_name}"] >= 4
)
result_df_answers_only["number_of_steps"] = result_df_answers_only["tools_used"].apply(
    lambda x: len(x) + 1
)
aggregated_resuts = result_df_answers_only.groupby(["agent_name", "task", "correct_answer"]).agg(
    {"number_of_steps": "mean"}
)

In [ ]:
px.bar(
    aggregated_resuts.reset_index(),
    x="task",
    y="number_of_steps",
    facet_col="agent_name",
    color="correct_answer",
    title="Mean Number of Steps to Solve a Task",
    barmode="group",
    labels={
        "agent_name": "Agent",
        "number_of_steps": "Mean Number of Steps",
        "correct_answer": "Answer correctness",
        "task": "Task",
    },
    width=1500,
)

Here we can see that on average, __tasks that failed have a higher number of steps taken than successful tasks__: taking many steps seems to be an indication that the agent is trying wrong directions to solve the problem.

### Display Prometheus vs GPT4

In [ ]:
res = pd.read_json("output/GPT4.json")
res = res.loc[~res["eval_score_human"].isnull()]
res = res[["eval_score_human", "eval_score_GPT4", "eval_score_Prometheus-13B-v1.0"]].reset_index(
    drop=True
)

In [ ]:
fig = px.bar(
    res,
)
fig.update_layout(
    width=1100,
    height=300,
    barmode="group",
    yaxis_range=[0.5, 5.5],
    yaxis_title="Score",
    yaxis=dict(
        tickmode="array",
        tickvals=[1, 3, 5],
    ),
)
fig.show()

In [ ]:
print("Proportion of cases matching human eval:")
print(
    (res["eval_score_human"] == res["eval_score_Prometheus-13B-v1.0"]).mean().round(3),
    "for Prometheus",
)
print((res["eval_score_human"] == res["eval_score_GPT4"]).mean().round(3), "for GPT4")

We could not use Prometheus-13B for this evaluation given its high rate of error.

In [ ]:
from langchain import hub
from langchain.agents import AgentExecutor, load_tools
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import (
    ReActJsonSingleInputOutputParser,
)
from langchain.tools.render import render_text_description
from langchain_community.utilities import SerpAPIWrapper

# setup tools

# setup ReAct style prompt
prompt = hub.pull("hwchase17/react-json")

In [ ]:
prompt